### 1. Import Dependencies

In [1]:
import os
# Open ai gym allows us to build environments and work with pre-existing environments
import gym
# stable_baselines : allows to vectorize environment, train RL agents on multiple environments at the same time.

# Proximal Policy Optimization algorithm combines ideas from AC2 (having multiple workers)
# and TRPO(it uses a trust region to improve the actor)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
# evaluate_policy : helps to test out how model is actually performing
from stable_baselines3.common.evaluation import evaluate_policy

### 2. Load Environment


In [2]:
# Mapping to the pre-installed openai gym enviroments
environment_name = 'CartPole-v1'
# Making environment
env = gym.make(environment_name)

In [3]:
environment_name

'CartPole-v1'

In [4]:
# Testing out CarPole env 5 times
episodes = 5  # (think of episode as one full game within the environemnt)
for episode in range(1,episodes+1):
    # Return initial set of observations : Agent, Action, Environment, Reward
    state = env.reset()
    # These obs will be passed to reinforcement learning agent to determine best type of action that is able to maximize ythe reward
    done  = False # whether or not episode is done
    score = 0 
    
    while not done:
        env.render() # allows to view the graphical representation of the environment
        
        # env.reset()  : reset the environment and obtain the initial observation
        # env.step()  : apply an action to the environemnt
        #env.close   : close down the render frame
        
        # Generating a random action (0 or 1)
        action = env.action_space.sample()
        # Passing through random action returns next set of observations and reward
        n_state, reward, done, info,_ = env.step(action)
        # Accumulating our reward
        score += reward
    print('Episode:{} Score:{}'.format(episode,score))
env.close()

Episode:1 Score:30.0
Episode:2 Score:11.0
Episode:3 Score:16.0
Episode:4 Score:23.0
Episode:5 Score:13.0


/Users/ritikagupta/anaconda3/lib/python3.11/site-packages/gym/envs/classic_control/cartpole.py:211: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(
/Users/ritikagupta/anaconda3/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [5]:
env.reset()

(array([-0.01729826, -0.00530884,  0.02909031, -0.04429241], dtype=float32),
 {})

In [6]:
env.observation_space.sample()

array([-2.6321878e+00, -3.0543664e+38, -6.5467753e-03,  2.8911070e+38],
      dtype=float32)

### Understanding the Environment
![Cart Pole](CartPole.png)


In [7]:
env.action_space

Discrete(2)

In [8]:
env.action_space.sample()

0

In [9]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [10]:
env.observation_space.sample()

array([-3.5914617e+00, -2.3257383e+38,  2.0571604e-01, -1.2499575e+38],
      dtype=float32)

### 3. Train an RL Model
![Train](Train.png)


* Model Free RL only uses current state values to try to make a prediction.
* Model Based RL try to make prediction about the future state of the model to try to generate best possible action.

#### Choosing Algorithms

* Action Space:
    - Discrete Single Process    : DQN
    - Discrete Multi Processed   : PPO or A2C
    - Continuous Single Process  : sac or TD3
    - Continuous Multi Processed : PPO or A2C

![Algorithm](Algorithms.png)

#### Understanding Training Metrics

* Evaluation Metrics:
  Ep_len_mean, ep_rew_mean

* Time Metrics:
Eps, iterations, time_elapsed, total_timestamps

* Loss Metrics:
Entropy_loss, policy_loss, value_loss

* Other Metrics:
Explained_variance, learning_rate, n_updates



In [11]:
# craete directories first
log_path = os.path.join('Training','Logs')

In [12]:
log_path

'Training/Logs'

In [13]:
env = gym.make(environment_name)
# Wrapping env inside DummyVecEnv
# Allow us to work with env wrapped inside dummy vectorized env
env = DummyVecEnv([lambda:env])
#Mlp : multi Layer Perceptron
# An agent's policy is a rule which tels it how to operate in the environment
# Stable Baseline3 has : MlpPolicy, CnnPolicy, MultiInputPolicy
model = PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path)

Using cpu device


/Users/ritikagupta/anaconda3/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [15]:
model.learn(total_timesteps=20000)

Logging to Training/Logs/PPO_2
-----------------------------
| time/              |      |
|    fps             | 4000 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 2643        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.005019945 |
|    clip_fraction        | 0.0392      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.558      |
|    explained_variance   | 0.856       |
|    learning_rate        | 0.0003      |
|    loss                 | 2.41        |
|    n_updates            | 110         |
|    policy_gradient_loss | -0.00663    |
|    value_loss           | 27.8        |
-----------------------------------------
---

### 4. Save and Reload Model

In [16]:
PPO_Path = os.path.join('Training','Saved Models','PPO_Model_Cartpole')

In [17]:
model.save(PPO_Path)

In [18]:
del model

In [20]:
PPO_Path

'Training/Saved Models/PPO_Model_Cartpole'

In [21]:
model.learn(total_timesteps=1000)

Logging to Training/Logs/PPO_3
-----------------------------
| time/              |      |
|    fps             | 3839 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------


In [19]:
# Reloading the model
model = PPO.load(PPO_Path,env=env)

/Users/ritikagupta/anaconda3/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


### 5. Evaluation

In [22]:
# PPO model is considered solved if we get an average score of 200 or higher
evaluate_policy(model,env,n_eval_episodes=10,render=True)

# CartPole reward is calculated as 1 point for every step that the pole remains upright.
# with a max of 200 steps.
# If pole is more than 15 degrees from vertical or the cart moves more than 2.4 units from center the episode ends.

/Users/ritikagupta/anaconda3/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/Users/ritikagupta/anaconda3/lib/python3.11/site-packages/stable_baselines3/common/vec_env/base_vec_env.py:243: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(500.0, 0.0)

### 6. Test the Model

In [24]:

episodes = 5 
for episode in range(1,episodes+1):

    obs = env.reset()
    done  = False 
    score = 0 
    
    while not done:
        env.render() # using the model
        action, _ = model.predict(obs) 
        obs, reward, done, info= env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode,score))
env.close()

Episode:1 Score:[500.]
Episode:2 Score:[500.]
Episode:3 Score:[500.]
Episode:4 Score:[500.]
Episode:5 Score:[500.]


In [25]:
#Get observations for our observation space
obs = env.reset()

In [29]:
action,_= model.predict(obs)

In [30]:
env.action_space.sample()

0

In [31]:
env.step(action) # By keeping our pole in an upright position and not letting it fall we accumulate a value of 1 every single time

(array([[ 0.04092013,  0.20921932,  0.04348042, -0.23916155]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{'TimeLimit.truncated': False}])

### 7. Viewing Logs in Tensorboard

In [32]:
training_log_path = os.path.join(log_path,'PPO_2')

In [33]:
training_log_path

'Training/Logs/PPO_2'

In [38]:
#tensorboard --logdir='Training/Logs/PPO_2' # run in terminal

Core Metrics to look at:
* Average Reward
* Average Episode length

Training Strategies
* Train for longer
* Hyperparameter Tuning
* Try different algorithms

### 8. Adding a callback to the training Stage

In [39]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [40]:
save_path = os.path.join('Training','Saved Models')

In [50]:
# Setting up stop training on Reward Threshold Callback: Stop training once we pass certain reward threshold
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200,verbose=1)
eval_callback = EvalCallback(env,
                             # everytime there's new best model it will run stop callback
                            callback_on_new_best=stop_callback,
                            # how frequently we want to run our eval callback
                             eval_freq=2000,
                            # After every 10000 runs check whether we passed the threshold if it has stop the training and save that to best model
                            best_model_save_path=save_path,
                            verbose=1)


In [51]:
model = PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path)

Using cpu device


In [52]:
model.learn(total_timesteps=20000,callback=eval_callback)

Logging to Training/Logs/PPO_7
Eval num_timesteps=2000, episode_reward=10.00 +/- 0.00
Episode length: 10.00 +/- 0.00
---------------------------------
| eval/              |          |
|    mean_ep_length  | 10       |
|    mean_reward     | 10       |
| time/              |          |
|    total_timesteps | 2000     |
---------------------------------
New best mean reward!
-----------------------------
| time/              |      |
|    fps             | 3945 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
Eval num_timesteps=4000, episode_reward=213.00 +/- 81.98
Episode length: 213.00 +/- 81.98
----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 213        |
|    mean_reward          | 213        |
| time/                   |            |
|    total_timesteps      | 4000       |
| train/                  |            |
|    approx_kl            | 0.008395

### 9. Changing Policies

In [53]:
# Define new neural network architecture
# vf: value function
net_arch = [dict(pi=[128,128,128,128],vf=[128,128,128,128])]

In [54]:
model = PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path,policy_kwargs={'net_arch':net_arch})

Using cpu device


/Users/ritikagupta/anaconda3/lib/python3.11/site-packages/stable_baselines3/common/policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [55]:
model.learn(total_timesteps=20000,callback=eval_callback)

Logging to Training/Logs/PPO_8


/Users/ritikagupta/anaconda3/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=2000, episode_reward=282.40 +/- 178.07
Episode length: 282.40 +/- 178.07
---------------------------------
| eval/              |          |
|    mean_ep_length  | 282      |
|    mean_reward     | 282      |
| time/              |          |
|    total_timesteps | 2000     |
---------------------------------
New best mean reward!
Stopping training because the mean reward 282.40  is above the threshold 200


### 10. Using an Alternative Algorithm

In [56]:
from stable_baselines3 import DQN

In [57]:
model = DQN('MlpPolicy',env,verbose=1,tensorboard_log=log_path)

Using cpu device


In [58]:
model.learn(total_timesteps=20000)

Logging to Training/Logs/DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.95     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1432     |
|    time_elapsed     | 0        |
|    total_timesteps  | 106      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.554    |
|    n_updates        | 1        |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.917    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1757     |
|    time_elapsed     | 0        |
|    total_timesteps  | 175      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.526    |
|    n_updates        | 18       |
----------------------------------
----------------------------------
| rollout/            | 

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.417    |
| time/               |          |
|    episodes         | 76       |
|    fps              | 2777     |
|    time_elapsed     | 0        |
|    total_timesteps  | 1227     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0879   |
|    n_updates        | 281      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.395    |
| time/               |          |
|    episodes         | 80       |
|    fps              | 2784     |
|    time_elapsed     | 0        |
|    total_timesteps  | 1274     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0688   |
|    n_updates        | 293      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rat

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 152      |
|    fps              | 2781     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2056     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0163   |
|    n_updates        | 488      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 156      |
|    fps              | 2778     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2096     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0147   |
|    n_updates        | 498      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rat

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 228      |
|    fps              | 2750     |
|    time_elapsed     | 1        |
|    total_timesteps  | 2816     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00505  |
|    n_updates        | 678      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 232      |
|    fps              | 2698     |
|    time_elapsed     | 1        |
|    total_timesteps  | 2856     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00361  |
|    n_updates        | 688      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rat

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 304      |
|    fps              | 2670     |
|    time_elapsed     | 1        |
|    total_timesteps  | 3568     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00105  |
|    n_updates        | 866      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 308      |
|    fps              | 2670     |
|    time_elapsed     | 1        |
|    total_timesteps  | 3604     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0017   |
|    n_updates        | 875      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rat

...

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1596     |
|    fps              | 2637     |
|    time_elapsed     | 6        |
|    total_timesteps  | 16474    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.015    |
|    n_updates        | 4093     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1600     |
|    fps              | 2637     |
|    time_elapsed     | 6        |
|    total_timesteps  | 16510    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0152   |
|    n_updates        | 4102     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rat

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1672     |
|    fps              | 2626     |
|    time_elapsed     | 6        |
|    total_timesteps  | 17273    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0397   |
|    n_updates        | 4293     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1676     |
|    fps              | 2626     |
|    time_elapsed     | 6        |
|    total_timesteps  | 17314    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.02     |
|    n_updates        | 4303     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rat

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1748     |
|    fps              | 2628     |
|    time_elapsed     | 6        |
|    total_timesteps  | 18173    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0309   |
|    n_updates        | 4518     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1752     |
|    fps              | 2628     |
|    time_elapsed     | 6        |
|    total_timesteps  | 18242    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0167   |
|    n_updates        | 4535     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rat

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1824     |
|    fps              | 2628     |
|    time_elapsed     | 7        |
|    total_timesteps  | 18995    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0255   |
|    n_updates        | 4723     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1828     |
|    fps              | 2628     |
|    time_elapsed     | 7        |
|    total_timesteps  | 19044    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0046   |
|    n_updates        | 4735     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rat

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1900     |
|    fps              | 2628     |
|    time_elapsed     | 7        |
|    total_timesteps  | 19890    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0266   |
|    n_updates        | 4947     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1904     |
|    fps              | 2629     |
|    time_elapsed     | 7        |
|    total_timesteps  | 19979    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00861  |
|    n_updates        | 4969     |
----------------------------------
